In [144]:
import time
import requests
from requests.structures import CaseInsensitiveDict
import numpy as np
import pandas as pd
import json
import os
import math

# Load metadata

In [78]:
documents = []
data_file = "data/Aerothermodynamics.json"
text_files_folder = "data/Aerothermodynamics"
if os.path.exists(data_file):
    with open(data_file) as f:
        documents = json.load(f)
else:
  url = "https://ntrs.nasa.gov/api/citations/search?skip=25"

  headers = CaseInsensitiveDict()
  headers["accept"] = "application/json"
  headers["Content-Type"] = "application/json"

  data = {
    "q": "Aerothermodynamics",
    "page":{"size":100,"from":0}
  }

  i_resp = requests.post(url, headers=headers, data=json.dumps(data))
  documents += i_resp.json()["results"]
  for i in range(100, math.ceil(i_resp.json()["stats"]["total"]), 100):
    data["page"]["from"] = i
    resp = requests.post(url, headers=headers, data=json.dumps(data))
    documents += resp.json()["results"]

  with open("data/Aerothermodynamics.json", "w") as f:
    json.dump(documents, f, indent=4)

# Fetch text files

In [110]:
file_urls = ["https://ntrs.nasa.gov" + i for i in [i["downloads"][0]["links"].get("fulltext") for i in documents if i["downloads"]] if i]

[i["downloads"][0]["links"].get("fulltext") for i in documents if i["downloads"]]


file_urls = []
for doc in documents:
    if doc["downloads"]:
        if doc["downloads"][0]["links"].get("fulltext"):
            file_urls.append((
                "https://ntrs.nasa.gov" + doc["downloads"][0]["links"].get("fulltext"),
                doc["id"]
                ))
file_urls

for url in file_urls:
    resp = requests.get(url[0])
    with open(f"data/Aerothermodynamics/{url[1]}.txt", "wb") as f:
        f.write(resp.content)
        print("Guardado", url)
    remaining_requests = int(resp.headers['X-RateLimit-Remaining'])
    if remaining_requests < 10:
        print("Near rate limit, sleeping")
        time.sleep(10 * (10 - remaining_requests))

[('https://ntrs.nasa.gov/api/citations/19870013510/downloads/19870013510.txt',
  19870013510),
 ('https://ntrs.nasa.gov/api/citations/20020050403/downloads/20020050403.txt',
  20020050403),
 ('https://ntrs.nasa.gov/api/citations/20080002266/downloads/20080002266.txt',
  20080002266),
 ('https://ntrs.nasa.gov/api/citations/19720013221/downloads/19720013221.txt',
  19720013221),
 ('https://ntrs.nasa.gov/api/citations/19720013230/downloads/19720013230.txt',
  19720013230),
 ('https://ntrs.nasa.gov/api/citations/20180004050/downloads/20180004050.txt',
  20180004050),
 ('https://ntrs.nasa.gov/api/citations/19830016235/downloads/19830016235.txt',
  19830016235),
 ('https://ntrs.nasa.gov/api/citations/20080005556/downloads/20080005556.txt',
  20080005556),
 ('https://ntrs.nasa.gov/api/citations/20200001407/downloads/20200001407.txt',
  20200001407),
 ('https://ntrs.nasa.gov/api/citations/20060000042/downloads/20060000042.txt',
  20060000042),
 ('https://ntrs.nasa.gov/api/citations/19870020144